In [1]:
!pip install fastapi uvicorn pyngrok transformers torch accelerate

In [2]:
!pip install -U bitsandbytes

In [ ]:
!pip install nest-asyncio


In [ ]:
!ngrok config add-authtoken token


In [ ]:
!pip install fastapi uvicorn pyngrok transformers torch pydantic nest-asyncio

from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Apply nest_asyncio to avoid event loop issues
nest_asyncio.apply()

# Load model and tokenizer
model_name = "saadsohail/TinyLlama-math-riddle-solver"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Initialize FastAPI app
app = FastAPI()

# Request body schema
class RiddleRequest(BaseModel):
    riddle: str

@app.post("/generate")
async def generate_riddle_solution(request: RiddleRequest):
    riddle_prompt = f"""Below is a math riddle. Write the solution to the riddle.

### Riddle:
{request.riddle}

### Solution:
"""

    inputs = tokenizer(riddle_prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.1,
        num_return_sequences=1,
    )

    solution = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the part after "Solution:"
    if "Solution:" in solution:
        solution = solution.split("Solution:")[1].strip()

    return {"solution": solution}



# Start ngrok tunnel
public_url = ngrok.connect(7860).public_url
print(f"Public API URL: {public_url}")

# Run Uvicorn server
uvicorn.run(app, host="0.0.0.0", port=7860)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/

Public API URL: https://aa2c-34-16-237-195.ngrok-free.app


INFO:     Started server process [1473]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7860 (Press CTRL+C to quit)


INFO:     2407:d000:403:936:f871:55a8:e2be:e96:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     2407:d000:403:936:f871:55a8:e2be:e96:0 - "POST /generate HTTP/1.1" 200 OK
